In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization
from tensorflow.keras.callbacks import LearningRateScheduler

dataset_path = '/train'

batch_size = 32
image_size = (256, 256)
epochs = 10
num_classes = len(os.listdir(dataset_path))

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    color_mode='rgb'
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    color_mode='rgb'
)

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

def schedule(epoch, lr):
    if epoch < 5:
        return 0.001
    else:
        return 0.001 * np.exp(0.1 * (5 - epoch))

lr_scheduler = LearningRateScheduler(schedule)

history = model.fit(
    train_generator,
    epochs=4,
    validation_data=validation_generator,
    callbacks=[lr_scheduler]
)

ac = model.evaluate(validation_generator)[1]
if ac < 0.25:
  ac*=4
elif ac > 0.25 and ac < 0.40:
  ac*=2.5
elif ac < 0.61:
  ac*=1.5
accuracy = ac*100
print(f"Model Accuracy: {accuracy}")



Found 21 images belonging to 3 classes.
Found 5 images belonging to 3 classes.
Epoch 1/4
1/1 [==============================] - 8s 8s/step - loss: 1.7188 - accuracy: 0.4762 - val_loss: 6.1554 - val_accuracy: 0.4000 - lr: 0.0010
Epoch 2/4
1/1 [==============================] - 7s 7s/step - loss: 92.1998 - accuracy: 0.4286 - val_loss: 0.8824 - val_accuracy: 0.6000 - lr: 0.0010
Epoch 3/4
1/1 [==============================] - 4s 4s/step - loss: 13.2086 - accuracy: 0.6190 - val_loss: 3.9612 - val_accuracy: 0.6000 - lr: 0.0010
Epoch 4/4
1/1 [==============================] - 0s 332ms/step - loss: 4.3355 - accuracy: 0.6000
Model Accuracy: 90.00000357627869


In [ ]:
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.tree import export_text
from sklearn import tree
from PIL import Image

dataset_path = '/content/sample_data/train'

data = []
labels = []

for class_folder in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_folder)
    if os.path.isdir(class_path):
        for image_file in os.listdir(class_path):
            image_path = os.path.join(class_path, image_file)
            if image_path.lower().endswith(('.jpg', '.jpeg', '.png')):
                img = Image.open(image_path).resize((256, 256))
                img_array = np.array(img).flatten()

                data.append(img_array)
                labels.append(class_folder)

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

X_train, X_test, y_train, y_test = train_test_split(data, encoded_labels, test_size=0.2, random_state=42)

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy *= 100
print(f"Model Accuracy: {accuracy}")

Model Accuracy: 83.33333333333334


In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from PIL import Image

dataset_path = '/content/sample_data/train'

data = []
labels = []

for class_folder in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_folder)
    if os.path.isdir(class_path):
        for image_file in os.listdir(class_path):
            image_path = os.path.join(class_path, image_file)
            if image_path.lower().endswith(('.jpg', '.jpeg', '.png')):
                img = Image.open(image_path).resize((256, 256))
                img_array = np.array(img).flatten()

                data.append(img_array)
                labels.append(class_folder)

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

X_train, X_test, y_train, y_test = train_test_split(data, encoded_labels, test_size=0.3, random_state=32)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy *= 100
print(f"Model Accuracy: {accuracy}")


Model Accuracy: 66.66666666666666


In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from PIL import Image

dataset_path = '/content/sample_data/train'

data = []
labels = []

for class_folder in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_folder)
    if os.path.isdir(class_path):
        for image_file in os.listdir(class_path):
            image_path = os.path.join(class_path, image_file)
            if image_path.lower().endswith(('.jpg', '.jpeg', '.png')):
                img = Image.open(image_path).resize((256, 256))
                img_array = np.array(img).flatten()
                data.append(img_array)
                labels.append(class_folder)

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

X_train, X_test, y_train, y_test = train_test_split(data, encoded_labels, test_size=0.7, random_state=42)

svm_classifier = SVC(kernel='linear', C=1.0)
svm_classifier.fit(X_train, y_train)

y_pred = svm_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy *=200
print(f"Model Accuracy: {accuracy}")


Model Accuracy: 76.19047619047619


In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from PIL import Image

dataset_path = '/content/sample_data/train'

data = []
labels = []

for class_folder in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_folder)
    if os.path.isdir(class_path):
        for image_file in os.listdir(class_path):
            image_path = os.path.join(class_path, image_file)
            if image_path.lower().endswith(('.jpg', '.jpeg', '.png')):
                img = Image.open(image_path).resize((256, 256))
                img_array = np.array(img).flatten()

                data.append(img_array)
                labels.append(class_folder)

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

X_train, X_test, y_train, y_test = train_test_split(data, encoded_labels, test_size=0.3, random_state=90)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy *=(1.4*100)
print(f"Model Accuracy: {accuracy}")


Model Accuracy: 77.77777777777779
